# 1. Import Dependencies

In [1]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

# 2.Types of Spaces

In [2]:
Discrete(3)

Discrete(3)

In [3]:
Discrete(3).sample()#0 3 arası değer verir

0

In [4]:
Box(0,1,shape=(3,3))

Box(3, 3)

In [5]:
Box(0,1,shape=(3,3)).sample()

array([[0.43941042, 0.40097845, 0.91754013],
       [0.57950145, 0.42512247, 0.9853515 ],
       [0.91749656, 0.79901713, 0.27786884]], dtype=float32)

In [6]:
Tuple((Discrete(3), Box(0,1,shape=(3,3))))

Tuple(Discrete(3), Box(3, 3))

In [7]:
Tuple((Discrete(3), Box(0,1,shape=(3,3)))).sample()

(1,
 array([[0.47558898, 0.3124085 , 0.22419007],
        [0.4304858 , 0.57674485, 0.1130814 ],
        [0.65798616, 0.6950891 , 0.1922135 ]], dtype=float32))

In [8]:
Dict({'height':Discrete(3), 'speed':Box(0,1,shape=(3,3))})

Dict(height:Discrete(3), speed:Box(3, 3))

In [9]:
Dict({'height':Discrete(3), 'speed':Box(0,100,shape=(1,))}).sample()

OrderedDict([('height', 0), ('speed', array([40.954315], dtype=float32))])

In [10]:
MultiBinary(4)

MultiBinary(4)

In [11]:
MultiBinary(4).sample()

array([1, 0, 1, 0], dtype=int8)

In [12]:
MultiDiscrete([5,6,3])

MultiDiscrete([5 6 3])

In [13]:
MultiDiscrete([5,6,3]).sample()

array([1, 5, 1], dtype=int64)

In [14]:
for i in range(10):
    print(Discrete(3).sample()-1)

-1
-1
0
-1
1
-1
-1
-1
-1
0


# 3. Building an Environment
- Build an agent to give us the best shower possible
- Randomly temperature
- 37 and 39 degrees optimum temperature

In [15]:
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(low = np.array([0]), high = np.array([100]))
        self.state = 38 + random.randint(-3,3)
        self.shower_length = 60
        
    def step(self, action):
        #apply temperature adjust
        self.state += action-1# 0=> -1 temperature decrease 1=> 0 nothing happens 2=> 1 temperature increase  
        
        #decrease shower time
        self.shower_length -= 1
        
        #calculate reward
        if self.state >= 37 and self.state <= 39:
            reward = +1
        else:
            reward = -1
            
        if self.shower_length <= 0:
            done = True
        else:
            done = False
        
        info = {}
        
        return self.state, reward, done, info
    
    def render(self):
        #implement viz
        pass
    
    def reset(self):
        self.state = np.array([38 + random.randint(-3, 3)]).astype(float)
        self.shower_length = 60
        return self.state

In [16]:
shower = ShowerEnv()

# 4. Test Environment

In [17]:
episodes = 5
for episode in range(1, episodes+1):
    obs = shower.reset()
    done = False
    score = 0 
    
    while not done:
        shower.render()
        action = shower.action_space.sample()
        obs, reward, done, info = shower.step(action)
        #print(obs, action-1, reward, done, info)
        score+=reward
        
    print('Episode:{} Score:{}'.format(episode, score))
shower.close()

Episode:1 Score:-6
Episode:2 Score:-10
Episode:3 Score:8
Episode:4 Score:-16
Episode:5 Score:-60


# 5. Train Model

In [18]:
log_path = os.path.join('Training', 'Logs')
model = PPO('MlpPolicy', shower, verbose = 1, tensorboard_log = log_path)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [19]:
model.learn(total_timesteps=500000)

Logging to Training\Logs\PPO_6
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -24.8    |
| time/              |          |
|    fps             | 318      |
|    iterations      | 1        |
|    time_elapsed    | 6        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -32.9       |
| time/                   |             |
|    fps                  | 301         |
|    iterations           | 2           |
|    time_elapsed         | 13          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.011065196 |
|    clip_fraction        | 0.0597      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | -0.000335   |

In [29]:
episodes = 5
for episode in range(1, episodes+1):
    obs = shower.reset()
    done = False
    score = 0 
    
    while not done:
        shower.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = shower.step(action)
        #print(obs, action-1, reward, done, info)
        score+=reward
        
    print('Episode:{} Score:{}'.format(episode, score))
shower.close()

Episode:1 Score:60
Episode:2 Score:58
Episode:3 Score:58
Episode:4 Score:54
Episode:5 Score:56


# 6 Save Model

In [21]:
save_path = os.path.join('Training', 'Saved Models','Shower_Model_PPO')
model.save(save_path)

In [33]:
evaluate_policy(model, shower, n_eval_episodes = 10)

(12.0, 58.787753826796276)